In [24]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# Loading dataset fashionMNIST

In [25]:
def load_dataset():
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomCrop(28, padding=2),
        transforms.ToTensor()
    ])

    trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

    return trainset, testset

In [26]:
trainset, testset = load_dataset()
train_loader = DataLoader(trainset, batch_size=128, shuffle=False)

# Normalizing dataset

In [27]:
def calculate_mean_and_std(loader):
    channel_sum = torch.zeros(1)
    channel_squared_sum = torch.zeros(1)
    total_pixels = 0

    for images, _ in loader:
        batch_pixels = images.size(0) * images.size(2) * images.size(3)
        total_pixels += batch_pixels
        channel_sum += images.sum(dim=[0, 2, 3])
        channel_squared_sum += (images ** 2).sum(dim=[0, 2, 3])
        
    mean = channel_sum / total_pixels
    std = torch.sqrt(channel_squared_sum / total_pixels - mean ** 2)
    return mean, std


In [31]:
mean, std = calculate_mean_and_std(train_loader)
print(mean)

tensor([0.2795])


In [40]:
def load_full_dataset():
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomCrop(28, padding=2),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

    trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

    return trainset, testset

In [62]:
n_train, n_test = load_full_dataset()
n_train_loader = DataLoader(n_train, batch_size=128, shuffle=True)
n_test_loader = DataLoader(n_test, batch_size=128, shuffle=False)

In [64]:
for images, _ in n_train_loader:
    mean = images.mean(dim=[0, 2, 3])
    std = images.std(dim=[0, 2, 3])
    print(f"Перевірка нормалізації: середнє {mean}, стандартне відхилення {std}")
    break

Перевірка нормалізації: середнє tensor([0.1485]), стандартне відхилення tensor([1.0078])


# Model architecture

In [65]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(7 * 7 * 128, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 10)

        self.drop = nn.Dropout(p=0.2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Conv2D -> ReLU
        x = torch.relu(self.conv2(x))  # Conv2D -> ReLU
        x = self.pool1(x)              # MaxPooling2D
        
        x = torch.relu(self.conv3(x))  # Conv2D -> ReLU
        x = torch.relu(self.conv4(x))  # Conv2D -> ReLU
        x = self.pool2(x)              # MaxPooling2D
        
        x = self.flatten(x)           # Flatten
        x = torch.relu(self.fc1(x))
        x = self.drop(x)
        x = torch.relu(self.fc2(x))   # Dense -> ReLU
        x = torch.relu(self.fc3(x))   # Dense -> ReLU
        x = self.fc4(x)               # Dense
        return x

# Parameters

In [66]:
lr = 0.001
decay = 0.0001
epochs = 20
device = 'mps'

# Optimizer

In [67]:
model = CNN()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0)
criterion = nn.CrossEntropyLoss()

In [68]:
writer = SummaryWriter()

# Train and validate

In [69]:
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for i, (data, target) in enumerate(n_train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(n_train_loader)
    train_losses.append(avg_train_loss)

    writer.add_scalar('Train/Batch_Loss', loss.item(), epoch * len(n_train_loader) + i)


    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in n_test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            total_val_loss += loss.item()

            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)

    avg_val_loss = total_val_loss / len(n_test_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = 100.0 * correct / total
    val_accuracies.append(val_accuracy)
    
    writer.add_scalar('Train/Epoch_Loss', avg_train_loss, epoch)
    writer.add_scalar('Validation/Loss', avg_val_loss, epoch)
    writer.add_scalar('Validation/Accuracy', val_accuracy, epoch)

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

writer.close()

Epoch 1/20
  Train Loss: 0.7347
  Val Loss: 0.4329, Val Accuracy: 84.04%
Epoch 2/20
  Train Loss: 0.3698
  Val Loss: 0.3696, Val Accuracy: 87.05%
Epoch 3/20
  Train Loss: 0.3191
  Val Loss: 0.3701, Val Accuracy: 85.82%
Epoch 4/20
  Train Loss: 0.2807
  Val Loss: 0.3105, Val Accuracy: 88.95%
Epoch 5/20
  Train Loss: 0.2651
  Val Loss: 0.2901, Val Accuracy: 89.40%
Epoch 6/20
  Train Loss: 0.2494
  Val Loss: 0.2730, Val Accuracy: 90.35%
Epoch 7/20
  Train Loss: 0.2389
  Val Loss: 0.2603, Val Accuracy: 90.54%
Epoch 8/20
  Train Loss: 0.2294
  Val Loss: 0.2562, Val Accuracy: 90.98%
Epoch 9/20
  Train Loss: 0.2178
  Val Loss: 0.2457, Val Accuracy: 91.52%
Epoch 10/20
  Train Loss: 0.2095
  Val Loss: 0.2401, Val Accuracy: 91.19%
Epoch 11/20
  Train Loss: 0.2043
  Val Loss: 0.2437, Val Accuracy: 91.53%
Epoch 12/20
  Train Loss: 0.1917
  Val Loss: 0.2425, Val Accuracy: 91.60%
Epoch 13/20
  Train Loss: 0.1895
  Val Loss: 0.2391, Val Accuracy: 91.43%
Epoch 14/20
  Train Loss: 0.1850
  Val Loss: 0.